In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [3]:
individuo = Net()
red = Net()
varianzas = Net()
# Asegurar que varianzas tiene valores positivos (p.ej., valor absoluto de inicialización o constante)
for param in varianzas.parameters():
    param.data = param.data.abs()
    # Opcional: Escalar a un rango inicial deseado, e.g. llenar con 0.1
    # param.data.fill_(0.1)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307],
                         std=[0.3081])
])

train_dataset = MNIST(root='../data/MNIST', train=True, download=True, transform=transform)
test_dataset = MNIST(root='../data/MNIST', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
loss_fn = nn.NLLLoss()  # función de pérdida para probabilidades logarítmicas (F.nll_loss)

best_loss = float('inf')
best_accuracy = 0.0
# Estructura para registrar mejoras recientes (últimas 10 iteraciones)
improve_history = [False] * 10  # tamaño fijo 10

max_epochs = 1000

In [6]:
for epoch in range(1, max_epochs+1):
    # == Mutación: crear copia del individuo y mutarla con ruido N(0, σ) ==
    individuo = Net()  # nuevo modelo para el hijo
    individuo.load_state_dict(individuo.state_dict())  # copiar pesos actuales
    # Añadir ruido gaussiano según varianzas
    with torch.no_grad():
        for (param_c, param_sigma) in zip(individuo.parameters(), varianzas.parameters()):
            # sample Gaussian noise for each parameter tensor
            noise = torch.normal(mean=torch.zeros_like(param_c.data), std=param_sigma.data)
            param_c.add_(noise)  # in-place add noise

    # == Generar supermáscara que preserve el 10% de mayores pesos absolutos ==
    # Obtener todos los valores absolutos de pesos (incluyendo sesgos) en un solo tensor
    all_weights = []
    for param in individuo.parameters():
        all_weights.append(param.data.view(-1).abs())
    all_weights = torch.cat(all_weights)
    # calcular el cutoff de top 10% (en caso de número total no divisible por 10, tomamos approx)
    k = int(0.10 * all_weights.numel())
    if k < 1:
        k = 1
    # torch.topk devuelve los k mayores; el último de esos será el umbral mínimo
    topk_vals, _ = torch.topk(all_weights, k)
    threshold = topk_vals.min()  # valor más pequeño entre los top k (umbral)
    # Aplicar máscara de threshold a cada tensor de pesos del individuo (hijo)
    for param in individuo.parameters():
        mask = (param.data.abs() >= threshold)
        param.data.mul_(mask)  # poner a 0 los pesos fuera del top 10%

    # Copiar el individuo en la red de evaluación `red`
    red.load_state_dict(individuo.state_dict())

    # == Evaluar la red enmascarada (forward completo en MNIST) ==
    red.eval()  # modo evaluación (aunque no hay capas dropout/batchnorm en este caso)
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for X, y in train_loader:
            X, y = X, y  # (en caso de usar GPU: .to(device))
            output = red(X)             # forward pass
            loss = loss_fn(output, y)   # calcular pérdida batch
            total_loss += loss.item() * X.size(0)
            # calcular predicciones correctas en el batch
            preds = output.argmax(dim=1)
            total_correct += (preds == y).sum().item()
            total_samples += X.size(0)
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    # Registrar resultados de esta iteración
    improved = avg_loss < best_loss
    improve_history.append(improved)
    if len(improve_history) > 10:
        improve_history.pop(0)  # mantener tamaño 10

    # Selección: ¿mejoró la pérdida respecto al mejor hasta ahora?
    if improved:
        best_loss = avg_loss
        best_accuracy = accuracy
        individuo.load_state_dict(individuo.state_dict())  # aceptar al hijo como nuevo individuo (genotipo)
        # (El individuo ahora tiene muchos pesos en cero según la máscara aplicada)

    # Adaptación de varianza (a partir de epoch 11)
    if epoch >= 10:
        # calcular fracción de éxitos en últimas 10 iteraciones
        success_rate = improve_history.count(True) / len(improve_history)
        if success_rate < 0.2:  # menos de 1/5 mejoras
            for param_sigma in varianzas.parameters():
                param_sigma.data.mul_(0.82)   # reducir varianza
        elif success_rate > 0.2:  # más de 1/5 mejoras
            for param_sigma in varianzas.parameters():
                param_sigma.data.mul_(1/0.82) # aumentar varianza (≈1.22x)
        # (si es exactamente 0.2, no se cambia σ)

    # Imprimir progreso (opcional)
    print(f"Iteración {epoch} - Pérdida: {avg_loss:.4f}, Exactitud: {accuracy*100:.2f}%{' [MEJORA]' if improved else ''}")

    # Criterio de parada: alcanzar 50% de accuracy
    if accuracy >= 0.50:
        print(f"¡Detenido! Se alcanzó {accuracy*100:.2f}% de exactitud en la iteración {epoch}.")
        break

# Guardar el mejor individuo y la red original inicial
torch.save(individuo.state_dict(), "mejor_individuo.pth")
torch.save(red.state_dict(), "red_inicial.pth")


KeyboardInterrupt: 